In [63]:
#! /usr/bin/env python3
import os
import sys
import openai
import time
import json

key_file = 'key'
#blas_file = 'blas.txt'
todo_txt = 'testedf.json'

with open(key_file, "r") as f:
    openai.api_key = f.readline().strip('\n')

todo = None
tested = None
with open(todo_txt) as f:
    tested = json.load(f)
    todo      = {key:tested[key] for key in tested.keys() if tested[key] == 'false' and key[0] == 's'}
    f.close()


for working_on,i in zip(todo.keys(), range(len(todo.keys()))):
   
    query = f"""Show me example of the cblas functions {working_on} in c style, showing inputs, function call, and expected outputs formatted as c arrays.
    Represent matrices as single dimension arrays,and define constant values (ints, float, enums...) in function call.
    Explain also in a few words what the funcion do."""
    ans =  openai.Completion.create(model="text-davinci-003", prompt=query, temperature=0, max_tokens=1000)
    result = ans["choices"][0]["text"]

    with open(f"test_cases/{working_on}.txt", "w") as f:
        f.write(result)
    
    tested[working_on] = 'generated'
    with open(todo_txt, "w") as f:
        json.dump(tested, f)
    

    print(f"\r Generated {working_on}: {i+1} / {len(todo.keys())}", end='')
    
    time.sleep(30)     
    

 Generated strsm: 1 / 1

In [53]:
#! /usr/bin/env python3
import os
import sys
import openai
import time
import json
import re

todo_txt = 'testedf.json'
todo = None
tested = None
with open(todo_txt) as f:
    tested = json.load(f)
    todo      = {key:tested[key] for key in tested.keys() if tested[key] == 'generated'}
    f.close()


for working_on in todo.keys():
    lines = None
    with open(f"test_cases/{working_on}.txt") as f:
        lines = f.read()

    lines = lines.split("\n")

    # Remove comments
    lines = [line[0:(line.find('//') if '//' in line else len(line))] for line in lines]               
    
    # Remove types, converts enums
    reps = {
        '{': '[', '}':']', ';':',', 'double': '', 'float': '', 'int':'', 'cblas_': '', 'Cblas':'blas', '.0f':'.0'
    }
    for k,v in reps.items():
        lines = [line.replace(k,v) for line in lines]                  

    # modify call
    for i in range(len(lines)):                                        
        if working_on in lines[i]:
            lines[i] = lines[i].replace(working_on + '(', 'blas:run({' + working_on + ',')  
            lines[i] = lines[i].replace(')', '})') 

    # Replace enums with their values
    for i in range(len(lines)):
        if '=' in lines[i] and '[' not in lines[i] and len(lines[i]) - lines[i].find('=') > 5:
            var_value = lines[i][lines[i].find('='):-1]
            var_value = var_value.replace('=','')
            var_value = var_value.replace(',','')
            var_value = var_value.strip()

            var_name = lines[i][0:lines[i].find('=')].strip()
            print

            lines[i] = ''

            for j in range(len(lines)):
                lines[j] = lines[j].replace(var_name, var_value)

    # In array declaration, remove brackets and name from declaration; rename variables names to upper case.
    for i in range(len(lines)):
        if '=' in lines[i]:
            first_bracket = lines[i].find('[') 
            equal = lines[i].find('=')

            var_name = lines[i][0: min(first_bracket, equal) if first_bracket > 0 else equal].strip()
            upper_var = var_name[0].upper() + var_name[1::]

            lines[i] = upper_var + ' ' +lines[i][lines[i].find('=')::]

            for j in range(len(lines)):
                if working_on in lines[j] and '{' in lines[j]:
                    line = re.split('{|}', lines[j])
                    v    = [i.strip() for i in line[1].split(',')]
                    for i in range(len(v)):
                        v[i] = v[i] if v[i] != var_name else upper_var
                    
                    lines[j] = line[0] + '{' + ", ".join(v) + "}" + line[2]


    # For matrices and vectors, buid arrays
    lines = [l.strip() for l in lines]
    for i in range(len(lines)):
        if '[' in lines[i]:
            l = lines[i].split('=')
            if i == len(lines) - 1:
                lines[i] =  l[1][0:-1] + f' = chain:btl({working_on[0]}, ' + l[0].strip() + ').'
            else:
                lines[i] = l[0] + f'= blas:new(chain:ltb({working_on[0]}, ' + l[1][0:-1].strip() + ')),'
                    

    lines = ['    ' + line.strip() for line in lines if line.strip() != '']
    lines[-1] = lines[-1] if lines[-1][-1] != ',' else lines[-1][0:-1] + '.'

    lines = '\n'.join(lines)

    lines = f'-module({working_on}_test).\n-include_lib("eunit/include/eunit.hrl").\n\n{working_on}_test()->\n' + lines

    with open(f'to_check/{working_on}_test.erl', 'w') as f:
        f.write(lines)    

In [ ]:
## Generate Double from Single precision

import json

todo_txt = 'testedf.json'
todo = None
tested = None
with open(todo_txt) as f:
    tested = json.load(f)
    todo      = [key for key in tested.keys() if tested[key] == 'tested']
    f.close()

for key in todo:
    with open(f"tested_s/{key}_test.erl") as f:
        d_key = 'd' + key[1::]
        content = f.read()
        content = content.replace(key, d_key)
        content = content.replace("(s,", "(d,")

        with open(f'to_test_d/{d_key}_test.erl', 'w') as out:
            out.write(content)
            
        tested[key] = "generated_sd"
        with open(todo_txt, "w") as f_json:
            json.dump(tested, f_json)


In [13]:
import json

todo_txt = 'testedf.json'
todo = None
tested = None
with open(todo_txt) as f:
    tested = json.load(f)
    todo      = {key:tested[key] for key in tested.keys() if tested[key] != 'removed'}
    f.close()


hashed = {
    's': [ 'saxpy', 'scopy', 'sswap', 'sscal', 'sdot', 'srot',
 'srotmg', 'sgemv', 'sger', 'sgbmv', 'ssbmv', 'srotg',
 'snrm2', 'sasum', 'isamax', 'isamin', 'ismax', 'ismin',
 'srotm', 'sdsdot', 'dsdot', 'ssum',
 'strmv', 'strmm', 'strsv', 'strsm',
 'sgemm', 'sspmv', 'sspr', 'sspr2',
 'stpsv', 'stpmv', 'stbsv', 'stbmv'],

    'd': ['daxpy','dcopy','dswap','dscal','ddot','drot',
'drotg','drotm','drotmg','dgemv','dger','dgbmv',
'dnrm2','dasum','idamax','idamin','idmax','idmin',
'dsbmv','dsum','csrot',
'dtrmv','dtrsv','dtrmm','dtrsm',
'dgemm','dspmv','dspr','dspr2',
'dtpsv','dtpmv','dtbsv','dtbmv'],

    'c': ['caxpy', 'ccopy', 'cscal', 'cdotu', 'cgemv', 'cgeru', 
'icamax','icamin','icmax', 'icmin', 
'cgerc', 'cgbmv', 'cswap', 'csscal', 'cdotc',
'ctrmv', 'ctrsv', 'ctrmm', 'ctrsm', 
'cgemm', 'cgemm3m', 'csrot',
'ctpsv', 'ctpmv', 'ctbsv', 'ctbmv'],

    'z': ['zaxpy', 'zcopy', 'zswap', 'zscal', 'zdotu', 'zdotc', 
'dznrm2', 'dzasum', 'izamax', 'izamin', 'izmax', 'izmin', 
'zrotg', 'zgemv', 'zgeru', 'zgerc', 'zgbmv', 'dzsum', 
'ztrmv', 'ztrsv', 'ztrmm', 'ztrsm', 'zdscal', 'zdrot', 
'zgemm', 'zgemm3m', 'ztbmv', 
'ztpsv', 'ztbsv']
}

for key in ['z']:#,'c','d','z']:
    for k in tested.keys():
        if k[0] == key and k not in hashed[key]:
            print("case " + k+":")

case ztpmv:
case zhemv:
case zhpr:
case zhpr2:
case zhbmv:
case zhpmv:
case zher:
case zhemm:
case zherk:
case zher2k:
case zsymm:
case zsyrk:
case zsyr2k:
case zher2:


In [4]:
import json

todo = {
    'ctrsm': 'strsm',
    'ctrmm': 'strmm',
    'crotg': 'srotg',
    'cgbmv': 'sgbmv',
    'ctbmv': 'stbmv',
    'ctbsv': 'stbsv',
    'ctpmv': 'stpmv',
    'ctpsv': 'stpsv',
    'chemv': 'ssymv',
    'chpr': 'sspr',
    'chpr2': 'sspr2',
    'chbmv': 'shbmv',
    'chpmv': 'shpmv',
    'cher': 'ssyr',
    'chemm': 'ssymm',
    'cherk': 'ssyrk',
    'cher2k': 'ssyr2k',
    'csymm': 'ssymm',
    'csyrk': 'ssyrk',
    'csyr2k': 'ssyr2k',
    'cher2': 'ssyr2'
}

descriptions = {
    'ctrsm': 'Solves a system of linear equations with a triangular matrix and a complex matrix using back substitution.',
    'ctrmm': 'Performs a matrix-matrix multiplication between a complex matrix and a triangular complex matrix.',
    'crotg': 'Computes the parameters of a complex plane rotation, given the values of two complex numbers.',
    'cgbmv': 'Performs a matrix-vector multiplication between a general band matrix and a complex vector.',
    'ctbmv': 'Performs a matrix-vector multiplication between a triangular band matrix and a complex vector.',
    'ctbsv': 'Solves a system of linear equations with a triangular band matrix and a complex vector using back substitution.',
    'ctpmv': 'Performs a matrix-vector multiplication between a packed triangular matrix and a complex vector.',
    'ctpsv': 'Solves a system of linear equations with a packed triangular matrix and a complex vector using back substitution.',
    'chemv': 'Performs a matrix-vector multiplication between a complex Hermitian matrix and a complex vector.',
    'chpr': 'Performs a rank-1 update of a complex Hermitian packed matrix.',
    'chpr2': 'Performs a rank-2 update of a complex Hermitian packed matrix.',
    'chbmv': 'Performs a matrix-vector multiplication between a complex Hermitian band matrix and a complex vector.',
    'chpmv': 'Performs a matrix-vector multiplication between a complex Hermitian packed matrix and a complex vector.',
    'cher': 'Performs a rank-1 update of a complex Hermitian matrix.',
    'chemm': 'Performs a matrix-matrix multiplication between two complex matrices.',
    'cherk': 'Performs a rank-k update of a complex Hermitian matrix.',
    'cher2k': 'Performs a rank-2k update of a complex Hermitian matrix.',
    'csymm': 'Performs a matrix-matrix multiplication between a complex symmetric matrix and a complex matrix.',
    'csyrk': 'Performs a rank-k update of a complex symmetric matrix.',
    'csyr2k': 'Performs a rank-2k update of a complex symmetric matrix.',
    'cher2': 'Performs a rank-2 update of a complex Hermitian matrix.'
}

for complex_name, real_name in todo.items():
    try:
        data = None
        with open(f'tested_s/{real_name}_test.erl', 'r') as in_file:
            data = in_file.read()
        
        data = data.replace(real_name, complex_name)
        data = "% " + descriptions[complex_name] + "\n\n" + data

        with open(f'to_check_c/{complex_name}_test.erl', 'w') as out_file:
          out_file.write(data)
        
        with open(todo_txt,'rw') as json_file:
            tested = json.load(json_file)
            tested[key] = "generated_c"
            json.dump(tested, json_file)
    except:
        continue

In [ ]:
import json

todo_txt = 'testedf.json'
todo = None
tested = None
with open(todo_txt) as f:
    tested = json.load(f)
    todo      = (key for key in tested.keys())
    f.close()
